In [1]:
#spark.stop()
%run ../connection.python.py

In [2]:
#input
impression_path = 'hdfs:///STAGE/adform/table=Impression/Impression.parquet'
trackingpoint_path = 'hdfs:///STAGE/adform/table=Trackingpoint/Trackingpoint.parquet'

In [3]:
###############################
#loading impression_table dataframe
###############################

impression_df = spark.read.format('parquet').load(impression_path)
# impression_df.printSchema()
impression_df = impression_df.filter(impression_df.IsRobot == 'No')
impression_df = impression_df.filter(impression_df.CookieID != 0)
impression_df = impression_df.where("yyyymmdd <= 20180831 and yyyymmdd >= 20180301")
impression_df = impression_df.where("regexp_extract(unloadvars.visibility, '\"visible1\":(\\\w+)',1)='true'").persist()
# impression_df.write.parquet("/user/stefano.mazzucca/py_digitalizzazione/impression_filt.parquet",mode="overwrite")
impression_df.createOrReplaceTempView("impression")

In [6]:
#leggo tracking point per costruire coppie cookieid-skyid
trackingpoint_df= spark.read.format('parquet').load(trackingpoint_path)
# trackingpoint_df.printSchema()
trackingpoint_df = trackingpoint_df.filter(trackingpoint_df.IsRobot == 'No')\
                                   .filter(trackingpoint_df.IsNoRepeats == 'Yes')\
                                   .filter(trackingpoint_df.CookieID != 0)
trackingpoint_df.createOrReplaceTempView("trackingpoint")

In [7]:
#coppie cookieid-skyid 
cookieid_skyid_df = trackingpoint_df.select(regexp_extract("customvars.systemvariables", '"13":"(\w+)',1).alias("skyid"), 'cookieid', 'device-name', 'os-name', 'browser-name').where("customvars.systemvariables like '%13%' and skyid <> '' and length(skyid) = 48 and cookieid <> '0'").persist()
cookieid_skyid_df = cookieid_skyid_df.distinct()
cookieid_skyid_df.createOrReplaceTempView("cookieid_skyid")
# cookieid_skyid_df.printSchema()
cookieid_skyid_df.show(10,False)

+------------------------------------------------+--------------------+------------------+----------+----------------+
|skyid                                           |cookieid            |device-name       |os-name   |browser-name    |
+------------------------------------------------+--------------------+------------------+----------+----------------+
|d7d5740854b729074c0308f2aa645d04d8f0dcfedd526d83|-4222772562261638140|Mobile            |iPhone    |Mobile Safari 11|
|d57285008b7f3d875279310f70687f8fd6dd0101dd7ab938|-4786014879383636041|Mobile            |iPhone    |Mobile Safari 11|
|599f5439c366bff9cedf9bbf0dfb4502694521ecddc0f54f|6836384834405944840 |Desktop and Laptop|Windows 10|Chrome          |
|348fa7f49112b8a92447e663d0780b57ff075c95f37ede23|-6792752978129413992|Mobile            |Android   |Chrome Mobile   |
|febad0be14b4b574272599b4720d52df9c94260e61d2317d|-8045372053882171762|Mobile            |iPhone    |Mobile Safari 11|
|50e2b62530479d4267b682c09656d923bde7a8b06cde3c1

In [8]:
lista_cookie_df = spark.sql("select cookieid from cookieid_skyid group by 1").persist()
lista_cookie_df.show(10,False)

+--------------------+
|cookieid            |
+--------------------+
|-3698676388661340072|
|-6795603234760284634|
|-2630790933627959160|
|-7649843254493758918|
|-5802941301142972906|
|-246833466618035261 |
|-7354590537594738070|
|-1515748146956991741|
|-2808758924860257011|
|-6937652819403878963|
+--------------------+
only showing top 10 rows



In [9]:
# lista_cookie_df.printSchema()

In [10]:
lista_cookie_df.createOrReplaceTempView("lista_cookie")

In [11]:
imps_sel_col_df = impression_df.select('cookieid', 'device-name','os-name','browser-name','publisherdomain','timestamp','yyyymmdd','city','region','country').persist()
imps_sel_col_df.createOrReplaceTempView("imps_sel_col")
imps_sel_col_df.show(10,False)

+--------------------+------------------+-----------+-------------+----------------------+-------------------+--------+---------------+--------+-------+
|cookieid            |device-name       |os-name    |browser-name |publisherdomain       |timestamp          |yyyymmdd|city           |region  |country|
+--------------------+------------------+-----------+-------------+----------------------+-------------------+--------+---------------+--------+-------+
|5711547453355456937 |Desktop and Laptop|Windows 10 |Edge         |*.tiscali.it          |2018-06-29 11:22:55|20180629|Fiano Romano   |Roma    |Italy  |
|7007545821645591060 |Desktop and Laptop|Windows 10 |Chrome       |*.mediaset.it         |2018-06-29 11:18:40|20180629|Trieste        |Trieste |Italy  |
|4877002184412356781 |Mobile            |Android    |Chrome Mobile|*.adnxs.com           |2018-06-29 11:22:24|20180629|Arese          |Milano  |Italy  |
|-5837774770026123281|Mobile            |Android    |Chrome Mobile|*.ilfattoquotid

In [15]:
skyid_imps_prep_step0_df = spark.sql("select distinct * from imps_sel_col as t1 inner join lista_cookie as t2 on t1.cookieid = t2.cookieid").persist()
# skyid_imps_prep_step0_df = spark.sql("select * from imps_sel_col where cookieid in (select cookieid from lista_cookie)").persist()

# skyid_imps_prep_step0_df.printSchema()
skyid_imps_prep_step0_df.createOrReplaceTempView("skyid_imps_prep_step0")

In [20]:
skyid_imps_prep_step0_df.show(10,False)
skyid_imps_prep_step0_df.printSchema()
skyid_imps_prep_step0_df.createOrReplaceTempView("skyid_imps_prep_step0")

+--------------------+-----------+-------+-------------+--------------------------+-------------------+--------+-------------+------+-------+--------------------+
|cookieid            |device-name|os-name|browser-name |publisherdomain           |timestamp          |yyyymmdd|city         |region|country|cookieid            |
+--------------------+-----------+-------+-------------+--------------------------+-------------------+--------+-------------+------+-------+--------------------+
|-9223181771274367924|Mobile     |Android|Chrome Mobile|*.doubleclick.net         |2018-06-17 16:02:53|20180617|Aosta        |Aosta |Italy  |-9223181771274367924|
|-9223181771274367924|Mobile     |Android|Chrome Mobile|*.doubleclick.net         |2018-06-17 16:04:03|20180617|Aosta        |Aosta |Italy  |-9223181771274367924|
|-9223181771274367924|Tablet     |Android|Chrome       |*.coltivazionebiologica.it|2018-04-24 19:45:33|20180424|Gressan      |Aosta |Italy  |-9223181771274367924|
|-9223181771274367924|

In [21]:
skyid_imps_prep_df = spark.sql("select cookieid, yyyymmdd, `device-name`, `os-name`, `browser-name`, concat(`device-name`,`os-name`) as device_os, substr(cast(yyyymmdd as varchar(8)),1,6) as mese, weekofyear(to_date(timestamp)) as week, date_format(timestamp, 'EEEE') as weekday, hour(timestamp) as hour, case when `device-name`='Mobile' then 1 else 0 end as flag_mobile, case when `device-name`='Tablet' then 1 else 0 end as flag_tablet, case when `device-name`='Desktop and Laptop' then 1 else 0 end as flag_desktop, publisherdomain, timestamp, city, region, country from skyid_imps_prep_step0").persist()
skyid_imps_prep_df.printSchema()
skyid_imps_prep_df.createOrReplaceTempView("skyid_imps_prep")

AnalysisException: "Reference 'cookieid' is ambiguous, could be: cookieid#339L, cookieid#440L.; line 1 pos 7"

In [ ]:
#skyid_imps_prep_df = spark.sql ("select cookieid, yyyymmdd, `device-name`, `os-name`, `browser-name`, concat(`device-name`,`os-name`) as device_os,substr(cast(yyyymmdd as varchar(8)),1,6) as mese,weekofyear(to_date(timestamp)) as week,date_format(timestamp, 'EEEE') as weekday, hour(timestamp) as hour, case when `device-name`='Mobile' then 1 else 0 end as flag_mobile,case when `device-name`='Tablet' then 1 else 0 end as flag_tablet, case when `device-name`='Desktop and Laptop' then 1 else 0 end as flag_desktop,publisherdomain,timestamp,city,region,country from imps_sel_col where cookieid in (select cookieid from lista_cookie)").persist()
#skyid_imps_prep_df.printSchema()
#skyid_imps_prep_df.createOrReplaceTempView("skyid_imps_prep")

In [14]:
# skyid_imps_prep_df.show(100)
skyid_imps_prep_df.write.parquet("/user/stefano.mazzucca/py_digitalizzazione/skyid_imps_1.parquet",mode="overwrite")

KeyboardInterrupt: 

In [ ]:
#giorno
skyid_imps_aggr_step0_df = spark.sql("select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd from skyid_imps_prep group by 1,2,3,4,5").persist()
skyid_imps_aggr_step0_df.printSchema()
skyid_imps_aggr_step0_df.createOrReplaceTempView("skyid_imps_aggr_step0")

In [ ]:
skyid_imps_aggr_step0_df.show(100)

In [ ]:
#mese
skyid_imps_aggr_step1_df = spark.sql("select cookieid, `device-name`, `os-name`, `browser-name`, mese from skyid_imps_prep group by 1,2,3,4,5").persist()
skyid_imps_aggr_step1_df.printSchema()
skyid_imps_aggr_step1_df.createOrReplaceTempView("skyid_imps_aggr_step1")

In [ ]:
skyid_imps_aggr_step1_df.show(100)

In [ ]:
#week
skyid_imps_aggr_step2_df = spark.sql("select cookieid, `device-name`, `os-name`, `browser-name`, week from skyid_imps_prep group by 1,2,3,4,5").persist()
skyid_imps_aggr_step2_df.printSchema()
skyid_imps_aggr_step2_df.createOrReplaceTempView("skyid_imps_aggr_step2")

In [ ]:
skyid_imps_aggr_step2_df.show(100)

In [ ]:
#imps tot e per device type
skyid_imps_aggr_step3_df = spark.sql("select cookieid, `device-name`, `os-name`, `browser-name`, count(*) as imps_tot, sum(flag_mobile) as imps_mobile, sum(flag_tablet) as imps_tablet, sum(flag_desktop) as imps_desktop from skyid_imps_prep group by 1,2,3,4").persist()
skyid_imps_aggr_step3_df.printSchema()
skyid_imps_aggr_step3_df.createOrReplaceTempView("skyid_imps_aggr_step3")

In [ ]:
skyid_imps_aggr_step3_df.show(100)

In [ ]:
#skyid_imps_aggr_step4_df=spark.sql("select skyid,count (distinct mese) as num_mesi ,count (distinct week) as num_week, count (distinct(yyyymmdd)) as num_giorni from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd from  skyid_imps_aggr_step0) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) left join (select cookieid, `device-name`, `os-name`, `browser-name`, mese from  skyid_imps_aggr_step1) as c on (a.cookieid = c.cookieid and a.`device-name`=c.`device-name` and a.`os-name`=c.`os-name` and a.`browser-name`=c.`browser-name`) left join (select cookieid, `device-name`, `os-name`, `browser-name`, week from  skyid_imps_aggr_step2) as d on (a.cookieid = d.cookieid and a.`device-name`=d.`device-name` and a.`os-name`=d.`os-name` and a.`browser-name`=d.`browser-name`) group by 1").persist()
#skyid_imps_aggr_step4_df=skyid_imps_aggr_step4_df.na.fill(0)
#skyid_imps_aggr_step4_df.printSchema()
#skyid_imps_aggr_step4_df.createOrReplaceTempView("skyid_imps_aggr_step4")

In [ ]:
skyid_imps_aggr_step4a_df=spark.sql("select skyid,count (distinct(yyyymmdd)) as num_giorni from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd from  skyid_imps_aggr_step0) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step4a_df=skyid_imps_aggr_step4a_df.na.fill(0)
skyid_imps_aggr_step4a_df.printSchema()
skyid_imps_aggr_step4a_df.createOrReplaceTempView("skyid_imps_aggr_step4a")

In [ ]:
skyid_imps_aggr_step4a_df.printSchema()

In [ ]:
skyid_imps_aggr_step4a_df.show(100)

In [ ]:
skyid_imps_aggr_step4a_df.count()

In [ ]:
skyid_imps_aggr_step4b_df=spark.sql("select skyid,count (distinct mese) as num_mesi from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, mese from  skyid_imps_aggr_step1) as c on (a.cookieid = c.cookieid and a.`device-name`=c.`device-name` and a.`os-name`=c.`os-name` and a.`browser-name`=c.`browser-name`) group by 1").persist()
skyid_imps_aggr_step4b_df=skyid_imps_aggr_step4b_df.na.fill(0)
skyid_imps_aggr_step4b_df.printSchema()
skyid_imps_aggr_step4b_df.createOrReplaceTempView("skyid_imps_aggr_step4b")

In [ ]:
skyid_imps_aggr_step4b_df.show(100)

In [ ]:
skyid_imps_aggr_step4b_df.count()

In [ ]:
skyid_imps_aggr_step4c_df=spark.sql("select skyid ,count (distinct week) as num_week from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, week from  skyid_imps_aggr_step2) as d on (a.cookieid = d.cookieid and a.`device-name`=d.`device-name` and a.`os-name`=d.`os-name` and a.`browser-name`=d.`browser-name`) group by 1").persist()
skyid_imps_aggr_step4c_df=skyid_imps_aggr_step4c_df.na.fill(0)
skyid_imps_aggr_step4c_df.printSchema()
skyid_imps_aggr_step4c_df.createOrReplaceTempView("skyid_imps_aggr_step4c")

In [ ]:
skyid_imps_aggr_step4c_df.show(100)

In [ ]:
skyid_imps_aggr_step4c_df.count()

In [ ]:
#num device step 1
skyid_imps_aggr_step5_df = spark.sql("select cookieid, `device-name`, `os-name`, `browser-name`, max(flag_mobile) as flag_mobile, max(flag_tablet) as flag_tablet, max(flag_desktop) as flag_desktop from skyid_imps_prep group by 1,2,3,4").persist()
skyid_imps_aggr_step5_df.printSchema()
skyid_imps_aggr_step5_df.createOrReplaceTempView("skyid_imps_aggr_step5")

In [ ]:
skyid_imps_aggr_step5_df.show(100)

In [ ]:
skyid_imps_aggr_step6_df = spark.sql("select skyid,count(distinct device_os) as num_device,  count(distinct case when flag_mobile=1 then device_os  else null end) as num_mobile,count(distinct case when flag_tablet=1 then device_os  else null end) as num_tablet,count(distinct case when flag_desktop=1 then device_os else null end) as num_desktop  from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, flag_mobile, flag_tablet, flag_desktop, concat(`device-name`,`os-name`) as device_os from  skyid_imps_aggr_step5) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step6_df=skyid_imps_aggr_step6_df.na.fill(0)
skyid_imps_aggr_step6_df.printSchema()
skyid_imps_aggr_step6_df.createOrReplaceTempView("skyid_imps_aggr_step6")

In [ ]:
skyid_imps_aggr_step6_df.show(100)

In [ ]:
skyid_imps_aggr_step6_df.count()

In [ ]:
skyid_imps_aggr_step7_df=spark.sql("select skyid, sum(imps_tot) as imps_tot,sum(imps_mobile) as imps_mobile, sum(imps_tablet) as imps_tablet, sum(imps_desktop) as imps_desktop from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, imps_tot, imps_mobile, imps_tablet, imps_desktop from  skyid_imps_aggr_step3) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step7_df=skyid_imps_aggr_step7_df.na.fill(0)
skyid_imps_aggr_step7_df.printSchema()
skyid_imps_aggr_step7_df.createOrReplaceTempView("skyid_imps_aggr_step7")

In [ ]:
skyid_imps_aggr_step7_df.show(100)

In [ ]:
skyid_imps_aggr_step7_df.count()

In [ ]:
import pyspark.sql.functions as F
skyid_imps_prep_dow_df = skyid_imps_prep_df.withColumn('Sunday', F.expr("IF(weekday = 'Sunday', 1, 0)")).withColumn('Monday', F.expr("IF(weekday = 'Monday', 1, 0)")).withColumn('Tuesday', F.expr("IF(weekday = 'Tuesday', 1, 0)")).withColumn('Wednesday', F.expr("IF(weekday = 'Wednesday', 1, 0)")).withColumn('Thursday', F.expr("IF(weekday = 'Thursday', 1, 0)")).withColumn('Friday', F.expr("IF(weekday = 'Friday', 1, 0)")).withColumn('Saturday', F.expr("IF(weekday = 'Saturday', 1, 0)"))
skyid_imps_prep_dow_df.printSchema()
skyid_imps_prep_dow_df.createOrReplaceTempView("skyid_imps_prep_dow")

In [ ]:
skyid_imps_prep_dow_df.show(100)

In [ ]:
#weekday
skyid_imps_aggr_step8_df = spark.sql("select skyid,count(distinct case when Monday=1 then yyyymmdd else null end) as num_monday,  count(distinct case when Tuesday=1 then yyyymmdd else null end) as num_tuesday,count(distinct case when Wednesday=1 then yyyymmdd else null end) as num_wednesday,count(distinct case when Thursday=1 then yyyymmdd else null end) as num_thursday,count(distinct case when Friday=1 then yyyymmdd else null end) as num_friday,count(distinct case when Saturday=1 then yyyymmdd else null end) as num_saturday,count(distinct case when Sunday=1 then yyyymmdd else null end) as num_sunday from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday from  skyid_imps_prep_dow) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step8_df.printSchema()
skyid_imps_aggr_step8_df.createOrReplaceTempView("skyid_imps_aggr_step8")

In [ ]:
skyid_imps_aggr_step8_df.show(100)

In [ ]:
#weekday  imps
skyid_imps_aggr_step9_df = spark.sql("select skyid,sum(case when Monday=1 then 1 else 0 end) as num_imps_monday,  sum(case when Tuesday=1 then 1 else 0 end) as num_imps_tuesday,sum(case when Wednesday=1 then 1 else 0 end) as num_imps_wednesday,sum(case when Thursday=1 then 1 else 0 end) as num_imps_thursday,sum(case when Friday=1 then 1 else 0 end) as num_imps_friday,sum(case when Saturday=1 then 1 else 0 end) as num_imps_saturday,sum(case when Sunday=1 then 1 else 0 end) as num_imps_sunday from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday from  skyid_imps_prep_dow) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step9_df.printSchema()
skyid_imps_aggr_step9_df.createOrReplaceTempView("skyid_imps_aggr_step9")

In [ ]:
skyid_imps_aggr_step9_df.show(100)

In [ ]:
skyid_imps_prep_hour_df = skyid_imps_prep_df.withColumn('fascia_0_1', F.expr("IF(hour = 0, 1, 0)")).withColumn('fascia_1_2', F.expr("IF(hour = 1, 1, 0)")).withColumn('fascia_2_3', F.expr("IF(hour = 2, 1, 0)")).withColumn('fascia_3_4', F.expr("IF(hour = 3, 1, 0)")).withColumn('fascia_4_5', F.expr("IF(hour = 4, 1, 0)")).withColumn('fascia_5_6', F.expr("IF(hour = 5, 1, 0)")).withColumn('fascia_6_7', F.expr("IF(hour = 6, 1, 0)")).withColumn('fascia_7_8', F.expr("IF(hour = 7, 1, 0)")).withColumn('fascia_8_9', F.expr("IF(hour = 8, 1, 0)")).withColumn('fascia_9_10', F.expr("IF(hour = 9, 1, 0)")).withColumn('fascia_10_11', F.expr("IF(hour = 10, 1, 0)")).withColumn('fascia_11_12', F.expr("IF(hour = 11, 1, 0)")).withColumn('fascia_12_13', F.expr("IF(hour = 12, 1, 0)")).withColumn('fascia_13_14', F.expr("IF(hour = 13, 1, 0)")).withColumn('fascia_14_15', F.expr("IF(hour = 14, 1, 0)")).withColumn('fascia_15_16', F.expr("IF(hour = 15, 1, 0)")).withColumn('fascia_16_17', F.expr("IF(hour = 16, 1, 0)")).withColumn('fascia_17_18', F.expr("IF(hour = 17, 1, 0)")).withColumn('fascia_18_19', F.expr("IF(hour = 18, 1, 0)")).withColumn('fascia_19_20', F.expr("IF(hour = 19, 1, 0)")).withColumn('fascia_20_21', F.expr("IF(hour = 20, 1, 0)")).withColumn('fascia_21_22', F.expr("IF(hour = 21, 1, 0)")).withColumn('fascia_22_23', F.expr("IF(hour = 22, 1, 0)")).withColumn('fascia_23_0', F.expr("IF(hour = 23, 1, 0)"))
skyid_imps_prep_hour_df.printSchema()
skyid_imps_prep_hour_df.createOrReplaceTempView("skyid_imps_prep_hour")

In [ ]:
skyid_imps_prep_hour_df.show(100)

In [ ]:
#hour of day imps
skyid_imps_aggr_step10_df = spark.sql("select skyid,sum(case when fascia_0_1=1 then 1 else 0 end) as num_imps_0_1,   sum(case when fascia_1_2=1 then 1 else 0 end) as num_imps_1_2,sum(case when fascia_2_3=1 then 1 else 0 end) as num_imps_2_3, sum(case when fascia_3_4=1 then 1 else 0 end) as num_imps_3_4,sum(case when fascia_4_5=1 then 1 else 0 end) as num_imps_4_5,sum(case when fascia_5_6=1 then 1 else 0 end) as num_imps_5_6,sum(case when fascia_6_7=1 then 1 else 0 end) as num_imps_6_7,sum(case when fascia_7_8=1 then 1 else 0 end) as num_imps_7_8,sum(case when fascia_8_9=1 then 1 else 0 end) as num_imps_8_9,sum(case when fascia_9_10=1 then 1 else 0 end) as num_imps_9_10,sum(case when fascia_10_11=1 then 1 else 0 end) as num_imps_10_11, sum(case when fascia_11_12=1 then 1 else 0 end) as num_imps_11_12,sum(case when fascia_12_13=1 then 1 else 0 end) as num_imps_12_13,sum(case when fascia_13_14=1 then 1 else 0 end) as num_imps_13_14,sum(case when fascia_14_15=1 then 1 else 0 end) as num_imps_14_15,sum(case when fascia_15_16=1 then 1 else 0 end) as num_imps_15_16,sum(case when fascia_16_17=1 then 1 else 0 end) as num_imps_16_17,sum(case when fascia_17_18=1 then 1 else 0 end) as num_imps_17_18,sum(case when fascia_18_19=1 then 1 else 0 end) as num_imps_18_19,sum(case when fascia_19_20=1 then 1 else 0 end) as num_imps_19_20,sum(case when fascia_20_21=1 then 1 else 0 end) as num_imps_20_21, sum(case when fascia_21_22=1 then 1 else 0 end) as num_imps_21_22,   sum(case when fascia_22_23=1 then 1 else 0 end) as num_imps_22_23,   sum(case when fascia_23_0=1 then 1 else 0 end) as num_imps_23_0   from (select skyid, cookieid, `device-name`, `os-name`, `browser-name` from cookieid_skyid where length(skyid)=48) as a left join (select cookieid, `device-name`, `os-name`, `browser-name`, yyyymmdd, fascia_0_1,fascia_1_2,fascia_2_3,fascia_3_4,fascia_4_5,fascia_5_6,fascia_6_7,fascia_7_8,fascia_8_9,fascia_9_10,fascia_10_11,fascia_11_12,fascia_12_13,fascia_13_14,fascia_14_15,fascia_15_16,fascia_16_17,fascia_17_18,fascia_18_19,fascia_19_20,fascia_20_21,fascia_21_22,fascia_22_23,fascia_23_0 from  skyid_imps_prep_hour) as b on (a.cookieid = b.cookieid and a.`device-name`=b.`device-name` and a.`os-name`=b.`os-name` and a.`browser-name`=b.`browser-name`) group by 1").persist()
skyid_imps_aggr_step10_df.printSchema()
skyid_imps_aggr_step10_df.createOrReplaceTempView("skyid_imps_aggr_step10")

In [ ]:
skyid_imps_aggr_step10_df.show(100)

In [ ]:
skyid_imps_aggr_step6_df.write.parquet("201809_sm_skyid_imps_aggr_step6",mode="overwrite")

In [ ]:
skyid_imps_aggr1_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step6')
skyid_imps_aggr1_df.printSchema()
skyid_imps_aggr1_df.createOrReplaceTempView("skyid_imps_aggr1")

In [ ]:
skyid_imps_aggr_step7_df.write.parquet("201809_sm_skyid_imps_aggr_step7",mode="overwrite")

In [ ]:
skyid_imps_aggr2_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step7')
skyid_imps_aggr2_df.printSchema()
skyid_imps_aggr2_df.createOrReplaceTempView("skyid_imps_aggr2")

In [ ]:
skyid_imps_aggr_step4a_df.write.parquet("201809_sm_skyid_imps_aggr_step4a",mode="overwrite")

In [ ]:
skyid_imps_aggr3_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step4a')
skyid_imps_aggr3_df.printSchema()
skyid_imps_aggr3_df.createOrReplaceTempView("skyid_imps_aggr3")

In [ ]:
skyid_imps_aggr_step4b_df.write.parquet("201809_sm_skyid_imps_aggr_step4b",mode="overwrite")

In [ ]:
skyid_imps_aggr4_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step4b')
skyid_imps_aggr4_df.printSchema()
skyid_imps_aggr4_df.createOrReplaceTempView("skyid_imps_aggr4")

In [ ]:
skyid_imps_aggr_step4c_df.write.parquet("201809_sm_skyid_imps_aggr_step4c",mode="overwrite")

In [ ]:
skyid_imps_aggr5_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step4c')
skyid_imps_aggr5_df.printSchema()
skyid_imps_aggr5_df.createOrReplaceTempView("skyid_imps_aggr5")

In [ ]:
skyid_imps_aggr_step8_df.write.parquet("201809_sm_skyid_imps_aggr_step8",mode="overwrite")

In [ ]:
skyid_imps_aggr6_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step8')
skyid_imps_aggr6_df.printSchema()
skyid_imps_aggr6_df.createOrReplaceTempView("skyid_imps_aggr6")

In [ ]:
skyid_imps_aggr_step9_df.write.parquet("201809_sm_skyid_imps_aggr_step9",mode="overwrite")

In [ ]:
skyid_imps_aggr7_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step9')
skyid_imps_aggr7_df.printSchema()
skyid_imps_aggr7_df.createOrReplaceTempView("skyid_imps_aggr7")

In [ ]:
skyid_imps_aggr_step10_df.write.parquet("201809_sm_skyid_imps_aggr_step10",mode="overwrite")

In [ ]:
skyid_imps_aggr8_df= spark.read.format('parquet').load('hdfs:///user/silvia/201809_sm_skyid_imps_aggr_step10')
skyid_imps_aggr8_df.printSchema()
skyid_imps_aggr8_df.createOrReplaceTempView("skyid_imps_aggr8")

In [ ]:
skyid_imps_aggr_finale_df= spark.sql("select a.skyid, num_mesi , num_week, num_giorni, imps_tot, imps_mobile, imps_tablet, imps_desktop, num_device, num_mobile, num_tablet, num_desktop,num_monday,num_tuesday,num_wednesday,num_thursday,num_friday,num_saturday,num_sunday,num_imps_monday,num_imps_tuesday,num_imps_wednesday,num_imps_thursday,num_imps_friday,num_imps_saturday,num_imps_sunday,num_imps_0_1,num_imps_1_2,num_imps_2_3,num_imps_3_4,num_imps_4_5,num_imps_5_6,num_imps_6_7,num_imps_7_8,num_imps_8_9,num_imps_9_10,num_imps_10_11,num_imps_11_12,num_imps_12_13,num_imps_13_14,num_imps_14_15,num_imps_15_16,num_imps_16_17,num_imps_17_18,num_imps_18_19,num_imps_19_20,num_imps_20_21,num_imps_21_22,num_imps_22_23,num_imps_23_0 from (select a.skyid, num_mesi , num_week, num_giorni from skyid_imps_aggr3 as a join skyid_imps_aggr4 as b on a.skyid=b.skyid join skyid_imps_aggr5 as c on a.skyid=c.skyid) as a inner join (select skyid, num_device, num_mobile, num_tablet, num_desktop from skyid_imps_aggr1) as b on a.skyid=b.skyid inner join (select skyid, imps_tot, imps_mobile, imps_tablet, imps_desktop from skyid_imps_aggr2) as c on a.skyid=c.skyid inner join (select a.*, num_monday,num_tuesday,num_wednesday,num_thursday,num_friday,num_saturday,num_sunday,num_imps_monday,num_imps_tuesday,num_imps_wednesday,num_imps_thursday,num_imps_friday,num_imps_saturday,num_imps_sunday from skyid_imps_aggr8 as a join skyid_imps_aggr7 as b on a.skyid=b.skyid join skyid_imps_aggr6 as c on a.skyid=c.skyid) as d on a.skyid=d.skyid").persist()

In [ ]:
skyid_imps_aggr_finale_df.show(100)

In [ ]:
skyid_imps_aggr_finale_df.count()

In [ ]:
skyid_imps_aggr_finale_df.printSchema()
skyid_imps_aggr_finale_df.createOrReplaceTempView("skyid_imps_aggr_finale")

In [ ]:
skyid_imps_aggr2_finale_df= spark.sql("select *, num_mobile/num_device as perc_mobile, num_tablet/num_device as perc_tablet, num_desktop/num_device as perc_desktop, imps_mobile/imps_tot as perc_imps_mobile, imps_tablet/imps_tot as perc_imps_tablet, imps_desktop/imps_tot as perc_imps_desktop, imps_tot/num_giorni as num_imps_per_giorno, imps_tot/num_mesi as num_imps_per_mese, imps_tot/num_week as num_imps_per_week, num_monday/num_giorni as perc_monday, num_tuesday/num_giorni as perc_tuesday, num_wednesday/num_giorni as perc_wednesday, num_thursday/num_giorni as perc_thursday, num_friday/num_giorni as perc_friday, num_saturday/num_giorni as perc_saturday, num_sunday/num_giorni as perc_sunday, num_imps_monday/imps_tot as perc_imps_monday, num_imps_tuesday/imps_tot as perc_imps_tuesday, num_imps_wednesday/imps_tot as perc_imps_wednesday, num_imps_thursday/imps_tot as perc_imps_thursday, num_imps_friday/imps_tot as perc_imps_friday, num_imps_saturday/imps_tot as perc_imps_saturday, num_imps_sunday/imps_tot as perc_imps_sunday, num_imps_0_1/imps_tot as perc_imps_0_1,num_imps_1_2/imps_tot as perc_imps_1_2,num_imps_2_3/imps_tot as perc_imps_2_3,num_imps_3_4/imps_tot as perc_imps_3_4,num_imps_4_5/imps_tot as perc_imps_4_5,num_imps_5_6/imps_tot as perc_imps_5_6,num_imps_6_7/imps_tot as perc_imps_6_7,num_imps_7_8/imps_tot as perc_imps_7_8,num_imps_8_9/imps_tot as perc_imps_8_9,num_imps_9_10/imps_tot as perc_imps_9_10,num_imps_10_11/imps_tot as perc_imps_10_11,num_imps_11_12/imps_tot as perc_imps_11_12,num_imps_12_13/imps_tot as perc_imps_12_13,num_imps_13_14/imps_tot as perc_imps_13_14,num_imps_14_15/imps_tot as perc_imps_14_15,num_imps_15_16/imps_tot as perc_imps_15_16,num_imps_16_17/imps_tot as perc_imps_16_17,num_imps_17_18/imps_tot as perc_imps_17_18,num_imps_18_19/imps_tot as perc_imps_18_19,num_imps_19_20/imps_tot as perc_imps_19_20,num_imps_20_21/imps_tot as perc_imps_20_21,num_imps_21_22/imps_tot as perc_imps_21_22,num_imps_22_23/imps_tot as perc_imps_22_23,num_imps_23_0/imps_tot as perc_imps_23_0 from skyid_imps_aggr_finale").persist()

In [ ]:
skyid_imps_aggr2_finale_df.show(100)

In [ ]:
skyid_imps_aggr2_finale_df.count()

In [ ]:
skyid_imps_aggr2_finale_df.printSchema()
skyid_imps_aggr2_finale_df.createOrReplaceTempView("skyid_imps_aggr2_finale")

In [ ]:
spark.sql("select count(distinct skyid) from skyid_imps_aggr2_finale").show()

In [ ]:
skyid_imps_aggr2_finale_df.write.parquet("201809_sm_skyid_imps_aggr_finale",mode="overwrite")

In [ ]:
skyid_imps_aggr2_finale_df.show(10)

In [ ]:
skyid_imps_aggr2_finale_df.count()